In [1]:
import re
import urllib2
import urllib
from bs4 import BeautifulSoup
import requests
import random
import time
import random
import pandas as pd
import numpy as np
import bs4

In [17]:
def select_random_proxy(proxy_sf):
    row_num = random.randint(0, len(proxy_sf) - 1)
    proxy_ip = "http://{}:{}@{}:{}".format(proxy_sf.iloc[row_num]['user'],
                                               proxy_sf.iloc[row_num]['pass'],
                                               proxy_sf.iloc[row_num]['ip'],
                                               proxy_sf.iloc[row_num]['port'])
    proxy = {"http": proxy_ip, "https":proxy_ip}
    return proxy
proxies = pd.read_csv('usa_proxy.csv',sep=':')
proxies.head()

,ip,port,user,pass
0,107.150.65.139,80,scrpr,m4anth1365
1,107.150.65.55,80,scrpr,m4anth1365
2,181.177.113.4,80,scrpr,m4anth1365
3,198.144.182.158,80,scrpr,m4anth1365
4,23.108.252.144,80,scrpr,m4anth1365


In [18]:
def select_random_UserAgent(UseragentList):
    row_num = random.randint(0, len(UseragentList) - 1)
    return UseragentList['User-agent'].iloc[row_num]

In [19]:
List=pd.read_csv('useragent - Sheet1 (1).csv')
List.head()

,User-agent
0,Google
1,Chrome
2,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...
3,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1...
4,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...


In [20]:
headers = {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
           "accept-encoding": "gzip, deflate, br",
           "accept-language": "en-GB,en-US;q=0.9,en;q=0.8,pl;q=0.7",
           "cache-control": "max-age=0",
           "referer": "www.amazon.com",
           "upgrade-insecure-requests":"1",
           "user-agent": select_random_UserAgent(List)}

In [21]:
def get_product_list(page,base_url,Category):

    FinalList=[]
    for i in range(2,page):
        aa=0
        while aa<7:
            try:
                page_url = base_url.format(i)

                time.sleep(random.randint(2,8))
                headers['User-agent']=select_random_UserAgent(List)
                #headers['referer']= base_url.format(i)
                print page_url
                res = requests.get(page_url,headers=headers,proxies=select_random_proxy(proxies))
                print str(res.status_code)
                if '403' in str(res.status_code) or '503' in str(res.status_code):
                    time.sleep(random.randint(2,20))
                    aa=aa+1
                else:
                    soup = bs4.BeautifulSoup(res.text,'html5lib')
                    Parent = soup.find_all(True,{"class":"s-result-item"})
                    #print(Parent)
                    for product in Parent:
                        PName=product.find("a",{"class":"a-text-normal"})
                        if PName:
                            PName=PName.text.strip()
                            print PName
                        else:
                            PName=None
                        PUrl=product.find("a",{"class": "a-link-normal a-text-normal"})
                        if PUrl:
                            PUrl=PUrl['href']
                            #print PUrl
                        else:
                            PUrl=None
                        Pprice=product.find("span",{"class":"a-offscreen"})
                        if Pprice:
                            Pprice=Pprice.text.strip()
                            #print Pprice
                        else:
                            Pprice=None
                        star_rating=product.find("span",{"class":"a-icon-alt"})
                        if star_rating:
                            star_rating=star_rating.text.strip()
                            #print star_rating
                        else:
                            star_rating=None
                        
                        product_id=product['data-asin']
                        #print product_id
                        if product_id:
                            product_id=product_id.strip()
                            print product_id
                        else:
                            product_id=None
                    
                        try:
                            reviews=product.find_all('a')
                            for count_a in reviews:
                                reviews_url=count_a['href']
                                #print reviews_url, 'customerReviews'.lower() in reviews_url.lower()
                                if 'customerReviews'.lower() in reviews_url.lower():
                                    review = count_a.text.replace(',','').replace('\n','')
                                    break
                                else:
                                    review = None
                        except:
                            review=None
                        #print review
                        Pimage=product.find("img",{"class":"s-image"})
                        if Pimage:
                            Pimage=Pimage['src']
                            #print Pimage
                        else:
                            Pimage=None

                        source = {'product_name':PName,
                                  'product_url':"https://www.amazon.com"+PUrl,
                                  'price':Pprice,
                                  'image_url':Pimage,
                                  'review_count':review,
                                  'star_rating':star_rating,
                                  'source_product_id':product_id}

                        FinalList.append(source)
                    aa=7
                    
            except Exception as ex:
                print('Error', ex)
                aa=aa+1
        temp=pd.DataFrame(FinalList)
        temp.to_csv(Category+'.csv',encoding='utf-8')
    print('Done')
    return FinalList

In [22]:
CategoryList=pd.read_csv('Amazon.com_smartphones.csv')

for cat in range(len(CategoryList)):
    
    Category=CategoryList['Category'].iloc[cat]
    CatList=CategoryList['Category_Link'].iloc[cat]
    
    base_url=CatList[:-1]+'{}'
    print base_url.format(2)
    
    ##### Enter Code of Finding Page No.
    headers['User-agent']=select_random_UserAgent(List)
    r=requests.get(base_url.format(2),headers=headers,proxies=select_random_proxy(proxies))
    soup = bs4.BeautifulSoup(r.text,'html5lib')
    inforbar=soup.find("span",{"data-component-type":"s-result-info-bar"})
    #print inforbar
    page=inforbar.find("div",{"class":"a-section a-spacing-small a-spacing-top-small"})
    count_nums = re.findall('\d+', page.text.replace(',',''))
    if(len(count_nums)==0):
        pages=100
    else:
        total_count = int(count_nums[-1])
        print total_count
        pages = total_count / 24 + bool(total_count%24)
        
    print(pages)
    CatData=get_product_list(pages,base_url,Category)
    CatData=pd.DataFrame(CatData)
    CatData.to_csv(Category+'_ProductList.csv',encoding='utf-8')
    print("Done for {}".format(Category))
    
print('Done')

https://www.amazon.com/s?i=mobile&rh=n%3A2335752011%2Cn%3A2335753011%2Cn%3A7072561011&page=2
7000
292
https://www.amazon.com/s?i=mobile&rh=n%3A2335752011%2Cn%3A2335753011%2Cn%3A7072561011&page=2
200
Samsung Galaxy S7 G930A 32GB Black Onyx - Unlocked GSM (Renewed)
B01F482BTK
Apple iPhone 6S, Fully Unlocked, 64GB - Space Gray (Renewed)
B06XR8G1TX
Apple iPhone 6S, GSM Unlocked, 16GB - Rose Gold (Renewed)
B01J8PBEUM
Apple iPhone 6 16 GB Unlocked, Gold (Refurbished)
B01BI2XQ5O
Tracfone Alcatel MyFlip 4G Prepaid Flip Phone
B07GBRB55Z
Google Pixel GSM Unlocked (Renewed) (32GB, Gray)
B072N3GKSM
Apple iPhone X, Fully Unlocked, 64GB - Space Gray (Renewed)
B077578W38
Samsung Galaxy S9 Unlocked - 64gb - Midnight Black - US Warranty (Renewed)
B07C65XFBB
Apple iPhone 8, Fully Unlocked, 64GB - Gold (Renewed)
B0775717ZP
Huawei P30 Lite (128GB, 4GB RAM) 6.15" Display, AI Triple Camera, 32MP Selfie, Dual SIM Global 4G LTE GSM Factory Unlocked MAR-LX3A - International Version (Midnight Black)
B07PXSNRYY


KeyboardInterrupt: 

In [8]:
test_df=pd.read_csv('Smartphones_ProductList.csv')

In [ ]:
#("div",class_="td-post-content")

In [9]:
len(test_df)

5904

In [12]:
test_df

,Unnamed: 0,image_url,price,product_name,product_url,review_count,source_product_id,star_rating
0,0,https://m.media-amazon.com/images/I/71u5x5hyJN...,$165.99,Samsung Galaxy S7 G930A 32GB Black Onyx - Unlo...,https://www.amazon.com/Samsung-Galaxy-G930A-32...,666.0,B01F482BTK,3.4 out of 5 stars
1,1,https://m.media-amazon.com/images/I/81s7ZLOGOW...,$195.55,"Apple iPhone 6S, Fully Unlocked, 64GB - Space ...",https://www.amazon.com/Apple-iPhone-Fully-Unlo...,518.0,B06XR8G1TX,3.5 out of 5 stars
2,2,https://m.media-amazon.com/images/I/41jUosGQiD...,$150.00,"Apple iPhone 6S, GSM Unlocked, 16GB - Rose Gol...",https://www.amazon.com/Apple-iPhone-6S-Unlocke...,816.0,B01J8PBEUM,3.9 out of 5 stars
3,3,https://m.media-amazon.com/images/I/81O0Tu14ce...,$130.82,"Apple iPhone 6 16 GB Unlocked, Gold (Refurbished)",https://www.amazon.com/Apple-iPhone-Unlocked-C...,312.0,B01BI2XQ5O,3.3 out of 5 stars
4,4,https://m.media-amazon.com/images/I/71p+nb8nEe...,$18.90,Tracfone Alcatel MyFlip 4G Prepaid Flip Phone,https://www.amazon.com/Tracfone-Alcatel-MyFlip...,86.0,B07GBRB55Z,3.5 out of 5 stars
5,5,https://m.media-amazon.com/images/I/81o50fc16k...,$128.98,"Google Pixel GSM Unlocked (Renewed) (32GB, Gray)",https://www.amazon.com/Google-Pixel-Unlocked-C...,729.0,B072N3GKSM,3.4 out of 5 stars
6,6,https://m.media-amazon.com/images/I/719knfTwPv...,$654.99,"Apple iPhone X, Fully Unlocked, 64GB - Space G...",https://www.amazon.com/Apple-iPhone-Fully-Unlo...,126.0,B077578W38,4.0 out of 5 stars
7,7,https://m.media-amazon.com/images/I/41KOSseOOd...,$374.96,Samsung Galaxy S9 Unlocked - 64gb - Midnight B...,https://www.amazon.com/Samsung-Galaxy-Unlocked...,291.0,B07C65XFBB,3.8 out of 5 stars
8,8,https://m.media-amazon.com/images/I/61pRPj+-IY...,$389.99,"Apple iPhone 8, Fully Unlocked, 64GB - Gold (R...",https://www.amazon.com/Apple-iPhone-Fully-Unlo...,176.0,B0775717ZP,3.9 out of 5 stars
9,9,https://m.media-amazon.com/images/I/41+BKn-Pv1...,$307.99,"Huawei P30 Lite (128GB, 4GB RAM) 6.15"" Display...",https://www.amazon.com/Huawei-Display-Factory-...,18.0,B07PXSNRYY,4.0 out of 5 stars


In [24]:
data=test_df.drop_duplicates(['source_product_id'],keep='first')

In [25]:
len(data)

4770

In [23]:
5904-4770

1134

In [26]:
data.to_csv('smartphones_without_duplicates.csv',encoding='utf-8')